## Script to create CRC STR and Splice Event pairs that occur within the same gene. The correlation between the CRC STR mean length and the Splice Event PSI for tumor samples will be calculated

In [32]:
# TODO: Analyse mit SpliceSeq daten
# Imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr

# Read STR data
df_coad = pd.read_csv("/Users/aychaserradj/Desktop/ZHAW/Praktikum/data/COAD_pt.csv")
df_read = pd.read_csv("/Users/aychaserradj/Desktop/ZHAW/Praktikum/data/READ_pt.csv")
# Read STR anotation
df_str_annotated = pd.read_csv("/Users/aychaserradj/Desktop/ZHAW/Praktikum/data/tral_and_perf_panel_meta_info_updated.tsv", sep='\t')
df_str_annotated = df_str_annotated[['tmp_id', 'gene_names', 'gene_ensembl_ids']]

# Combine COAD and READ STR
df = df_read.append(df_coad)
# Calculate STR mean length
df['mean_length'] = (df['allele_a'] + df['allele_b']) / 2


# Create STR pivot table -> for each STR and tumor sample the table contains the STR mean length
pivot_df = pd.pivot_table(df, 
                        index='tmp_id', 
                        columns='sample', 
                        values=['mean_length'], 
                        aggfunc='mean',
                        fill_value=0)

# Set columns name
pivot_df.columns = [col[1] for col in pivot_df.columns]
pivot_df_copy = pivot_df.copy()
# Cut column names 
pivot_df_copy.columns = ['-'.join(col.split('-')[:4]) for col in pivot_df_copy.columns]
# Remove STR with same length over all tumor samples
pivot_df = pivot_df_copy.drop_duplicates(keep=False)
# Count zero values for each row
zero_counts = pivot_df.eq(0).sum(axis=1)
# Count non-zero values for each row
non_zero_counts = pivot_df.ne(0).sum(axis=1)
# Filter rows where the number of zeros is less than or equal to the number of non-zero values
pivot_df = pivot_df[(zero_counts <= non_zero_counts)]

# Indices to column
pivot_df.reset_index(inplace=True)
# Replace "_" by "-" in column names
pivot_df.rename(columns=lambda x: x.replace('_', '-'), inplace=True)
# Rename 'tmp-id' column
pivot_df.rename(columns={'tmp-id': 'tmp_id'}, inplace=True)
# Merge pivor_ df annotation file -> create annotated CRC STRs
pivot_annotated = pd.merge(pivot_df, df_str_annotated, on='tmp_id', how='inner')
# Relocate columns
cols = list(pivot_annotated.columns)
cols.insert(1, cols.pop(cols.index('gene_names')))
cols.insert(2, cols.pop(cols.index('gene_ensembl_ids')))
pivot_annotated = pivot_annotated[cols]
# Remove duplicated columns
pivot_annotated = pivot_annotated.loc[:, ~pivot_annotated.columns.duplicated()]
pivot_annotated

,tmp_id,gene_names,gene_ensembl_ids,TCGA-3L-AA1B-01A,TCGA-4N-A93T-01A,TCGA-4T-AA8H-01A,TCGA-5M-AAT4-01A,TCGA-5M-AAT5-01A,TCGA-5M-AAT6-01A,TCGA-5M-AATA-01A,...,TCGA-QG-A5YV-01A,TCGA-QG-A5YW-01A,TCGA-QG-A5YX-01A,TCGA-QG-A5Z1-01A,TCGA-QG-A5Z2-01A,TCGA-QL-A97D-01A,TCGA-RU-A8FL-01A,TCGA-SS-A7HO-01A,TCGA-T9-A92H-01A,TCGA-WS-AB45-01A
0,chr10_100042463,CPN1,ENSG00000120054,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,chr10_100069930,CPN1,ENSG00000120054,10,10,10,10,10,10,10,...,0,0,0,0,0,10,10,10,10,10
2,chr10_1000869,GTPBP4,ENSG00000107937,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
3,chr10_100152407,ERLIN1,ENSG00000107566,4,4,4,4,4,4,4,...,4,0,4,0,4,4,4,4,4,4
4,chr10_100218932,CHUK,ENSG00000213341,17,17,17,17,17,17,17,...,0,0,0,0,0,17,17,17,17,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32524,chrX_97247622,DIAPH2,ENSG00000147202,9,9,9,9,9,9,9,...,9,0,0,0,0,9,9,9,9,8
32525,chrX_97247846,DIAPH2,ENSG00000147202,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
32526,chrX_97429811,DIAPH2,ENSG00000147202,11,11,12,12,11,9,11,...,0,0,0,0,0,11,11,11,13,10
32527,chrX_9760652,GPR143,ENSG00000101850,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [56]:
# Import Splicing Event dataset
df_SplAdder = pd.read_csv("/Users/aychaserradj/Desktop/ZHAW/Praktikum/data/SplAdder_PSI_COAD_and_READ.tsv", sep='\t')
# Remove duplicated rows
df_SplAdder = df_SplAdder.drop_duplicates()

# Split ASE tumor samples names -> create same name structure like STR (to compare STR and ASE samples)
cols_to_modify = df_SplAdder.columns[15:]
df_SplAdder.rename(columns={col: '-'.join(col.split('-')[:4]) for col in cols_to_modify}, inplace=True)
# Remove duplicated columns
df_SplAdder = df_SplAdder.loc[:, ~df_SplAdder.columns.duplicated()]

# Columns of STR data
cols_str = pivot_df.columns[1:].tolist()
# Columns of ASE data
cols_ase = df_SplAdder.columns[15:].tolist()
# Get tumor samples that are in STR and ASE data
cols_str_ase = set(cols_ase).intersection(cols_str) # 356 common
# Convert set 'cols_str_ase' to list
cols_str_ase = list(cols_str_ase)

# Create STR with tumor samples and the first 3 columns of the original columns ("tmp_id","gene_names","gene_ensembl_ids")
pivot_annotated = pivot_annotated[list(pivot_annotated.columns[:3]) + cols_str_ase]
# Create ASE with tumor samples and the first 13 columns
df_SplAdder = df_SplAdder[list(df_SplAdder.columns[:15]) + cols_str_ase]
# Rename column 'GeneSymbol' in ASE data
df_SplAdder.rename(columns={'GeneSymbol': 'gene_names'}, inplace=True)
# count zero-values for each rows
ase_zero_counts = df_SplAdder.iloc[:, 15:].apply(lambda row: row.isin([0, np.nan]).sum(), axis=1)
# count non zero-values for each rows
ase_non_zero_counts = (df_SplAdder.iloc[:, 15:] != 0).sum(axis=1) & df_SplAdder.iloc[:, 15:].notna().sum(axis=1)
# Filter rows where the number of zeros is less than or equal 
df_SplAdder = df_SplAdder[(ase_zero_counts <= ase_non_zero_counts)]


In [57]:
df_SplAdder

,SpliceEvent,gene_names,GeneID,ChrStrand,SpliceType,EventRegion,AltRegion,AltRegionIsoName,SpliceInIsoName,SpliceOutIsoName,...,TCGA-DC-5869-01A,TCGA-EI-6512-01A,TCGA-G4-6317-01A,TCGA-CM-6676-01A,TCGA-AG-4021-01A,TCGA-AA-3675-01A,TCGA-A6-6654-01A,TCGA-D5-7000-01A,TCGA-CL-5917-01A,TCGA-DM-A1D0-01A
0,alt_3prime_100,AGRN,ENSG00000188157.9,chr1 | +,A3,986632:986749-986832:986883-986883:986905,986832:986905,NaN,"AGRN-001,AGRN-009",NaN,...,NaN,NaN,1.000,1.000,NaN,1.000,1.000,1.000,NaN,1.000
1,alt_3prime_100002,ZNF207,ENSG00000010244.12,chr17 | +,A3,30677216:30677345-30678804:30678816-30678816:3...,30678804:30678902,ZNF207-201 (E2),"ZNF207-003,ZNF207-201,ZNF207-001,ZNF207-002,ZN...",ZNF207-011,...,NaN,NaN,0.043,0.030,NaN,0.043,0.019,0.014,NaN,0.016
2,alt_3prime_100008,ZNF207,ENSG00000010244.12,chr17 | +,A3,30685521:30685660-30687616:30687620-30687620:3...,30687616:30687694,ZNF207-009 (E4),"ZNF207-003,ZNF207-005,ZNF207-001,ZNF207-002,ZN...",NaN,...,NaN,NaN,1.000,1.000,NaN,0.962,1.000,1.000,NaN,1.000
3,alt_3prime_100010,ZNF207,ENSG00000010244.12,chr17 | +,A3,30687769:30687784-30687910:30687931-30687931:3...,30687910:30687986,"ZNF207-201 (E4),ZNF207-001 (E5),ZNF207-003 (E5...","ZNF207-003,ZNF207-201,ZNF207-005,ZNF207-001,ZN...",NaN,...,NaN,NaN,1.000,1.000,NaN,1.000,1.000,1.000,NaN,1.000
4,alt_3prime_100023,ZNF207,ENSG00000010244.12,chr17 | +,A3,30694790:30695033-30696313:30696365-30696365:3...,30696313:30696473,"ZNF207-201 (E10),ZNF207-006 (E11),ZNF207-003 (...","ZNF207-003,ZNF207-201,ZNF207-005,ZNF207-001,ZN...",NaN,...,NaN,NaN,1.000,1.000,NaN,1.000,1.000,1.000,NaN,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65458,mutex_exons_8985,TYSND1,ENSG00000156521.9,chr10 | -,ME,71899109:71899897-71902520:71902609-71902423:7...,71902520:71902609,NaN,NaN,NaN,...,NaN,NaN,0.580,0.614,NaN,0.525,0.517,0.597,NaN,0.593
65459,mutex_exons_9211,ANXA7,ENSG00000138279.11,chr10 | -,ME,75156276:75156341-75156921:75157032-75157977:7...,75156921:75157032,"ANXA7-005 (E4),ANXA7-003 (E3),ANXA7-001 (E4),A...",NaN,ANXA7-201,...,NaN,NaN,0.873,0.858,NaN,0.846,0.833,0.776,NaN,0.830
65464,mutex_exons_9633,MMS19,ENSG00000155229.16,chr10 | -,ME,99236485:99236501-99236676:99236720-99236591:9...,99236676:99236720,NaN,NaN,MMS19-004,...,NaN,NaN,0.604,0.541,NaN,0.564,0.496,0.493,NaN,0.500
65465,mutex_exons_9823,USMG5,ENSG00000173915.8,chr10 | -,ME,105151947:105152040-105152138:105152223-105152...,105152138:105152223,NaN,NaN,NaN,...,NaN,NaN,0.459,0.491,NaN,0.483,0.484,0.529,NaN,0.486


In [60]:
# Merge annotated CRC STR and SplAdder ASE -> create pairs of ASE and STR that occur in the same gene
str_ase = pd.merge(pivot_annotated, df_SplAdder, on='gene_names', how='inner')
# Delete duplicated based on the column tmp_id', 'gene_ensembl_ids'
str_ase = str_ase.drop_duplicates(subset=['tmp_id', 'gene_ensembl_ids'])
str_ase

,tmp_id,gene_names,gene_ensembl_ids,TCGA-G4-6321-01A_x,TCGA-F5-6571-01A_x,TCGA-A6-5656-01A_x,TCGA-D5-6924-01A_x,TCGA-G4-6626-01A_x,TCGA-CK-5912-01A_x,TCGA-CK-6751-01A_x,...,TCGA-DC-5869-01A_y,TCGA-EI-6512-01A_y,TCGA-G4-6317-01A_y,TCGA-CM-6676-01A_y,TCGA-AG-4021-01A_y,TCGA-AA-3675-01A_y,TCGA-A6-6654-01A_y,TCGA-D5-7000-01A_y,TCGA-CL-5917-01A_y,TCGA-DM-A1D0-01A_y
0,chr10_1000869,GTPBP4,ENSG00000107937,4,4,4,4,4,4,4,...,NaN,NaN,1.000,1.000,NaN,1.000,1.000,0.989,NaN,1.000
2,chr10_1009084,GTPBP4,ENSG00000107937,4,0,0,4,4,4,4,...,NaN,NaN,1.000,1.000,NaN,1.000,1.000,0.989,NaN,1.000
4,chr10_1014160,GTPBP4,ENSG00000107937,9,9,0,9,9,9,9,...,NaN,NaN,1.000,1.000,NaN,1.000,1.000,0.989,NaN,1.000
6,chr10_996145,GTPBP4,ENSG00000107937,4,4,4,4,4,4,4,...,NaN,NaN,1.000,1.000,NaN,1.000,1.000,0.989,NaN,1.000
8,chr10_999123,GTPBP4,ENSG00000107937,3,3,3,3,3,3,3,...,NaN,NaN,1.000,1.000,NaN,1.000,1.000,0.989,NaN,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59036,chrX_85306276,POF1B,ENSG00000124429,4,4,0,4,4,4,4,...,NaN,NaN,0.879,0.933,NaN,0.995,0.825,0.878,NaN,0.948
59040,chrX_85308283,POF1B,ENSG00000124429,10,0,10,10,0,10,10,...,NaN,NaN,0.879,0.933,NaN,0.995,0.825,0.878,NaN,0.948
59044,chrX_85315745,POF1B,ENSG00000124429,9,9,9,9,9,9,9,...,NaN,NaN,0.879,0.933,NaN,0.995,0.825,0.878,NaN,0.948
59048,chrX_85351566,POF1B,ENSG00000124429,4,4,0,4,0,4,4,...,NaN,NaN,0.879,0.933,NaN,0.995,0.825,0.878,NaN,0.948


In [61]:
# Create data frame of STR and ASE pairs 
str_ase_pairs = str_ase[['tmp_id','SpliceEvent']]
str_ase_pairs

,tmp_id,SpliceEvent
0,chr10_1000869,exon_skip_38809
2,chr10_1009084,exon_skip_38809
4,chr10_1014160,exon_skip_38809
6,chr10_996145,exon_skip_38809
8,chr10_999123,exon_skip_38809
...,...,...
59036,chrX_85306276,alt_3prime_256518
59040,chrX_85308283,alt_3prime_256518
59044,chrX_85315745,alt_3prime_256518
59048,chrX_85351566,alt_3prime_256518


In [67]:
# Create empty data frame 
df_str_ase = pd.DataFrame(columns=['crc_str', 'splice_event', 'pearson_coefficient', 'p_value'])

# Iterate over len of STR-ASE pairs 
for i in range(len(str_ase_pairs)):
    # Select STR
    crc_str = pivot_annotated[pivot_annotated['tmp_id'] == str_ase_pairs['tmp_id'].iloc[i]].iloc[:,3:].iloc[0].tolist()
    # Select ASE
    ase = df_SplAdder[df_SplAdder['SpliceEvent'] == str_ase_pairs['SpliceEvent'].iloc[i]].iloc[:, 15:].iloc[0].tolist()
    # Create data frame of the selected STR and ASE
    data = {'crc_str': crc_str,'ase': ase}
    # Create a DataFrame
    df = pd.DataFrame(data)
    # Fill 'NaN' values with 0
    df = df.fillna(0)
    # Calculate pearson_coefficient and p_value for each STR-ASE pair
    correlation_coefficient, p_value = pearsonr(df['crc_str'], df['ase'])
    # Fill data frame
    str_ase = {'crc_str': str_ase_pairs['tmp_id'].iloc[i], 
                'splice_event': str_ase_pairs['SpliceEvent'].iloc[i],
                'pearson_coefficient': corr,
                'p_value': p_value}
    df_str_ase = df_str_ase.append(str_ase, ignore_index=True)
# Sort data frame by 'pearson_coefficient'
df_str_ase = df_str_ase.sort_values(by='pearson_coefficient', ascending=False)

In [69]:
df_str_ase.reset_index(inplace=True)
df_str_ase

,index,crc_str,splice_event,pearson_coefficient,p_value
0,5972,chr22_37650658,alt_3prime_167635,0.378467,1.440050e-13
1,3180,chr17_2690565,alt_3prime_112262,0.378300,1.478779e-13
2,3697,chr17_81996788,alt_3prime_111669,0.361331,2.026870e-12
3,9041,chr6_42564302,alt_3prime_215714,0.359698,2.586922e-12
4,5869,chr22_20564610,alt_3prime_166475,0.342188,3.243845e-11
...,...,...,...,...,...
10971,5293,chr1_6587557,intron_retention_527,-0.259046,7.220974e-07
10972,10591,chr9_85975268,intron_retention_129314,-0.260002,6.549357e-07
10973,275,chr10_63215242,alt_3prime_30845,-0.270480,2.190847e-07
10974,10580,chr9_82982786,intron_retention_132178,-0.276202,1.180899e-07
